In [64]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import t

df = pd.read_csv('Temp_MN.csv')

# Povprečje in standardni odklon
n = df.shape[0]
mu = df['TEMP'].mean()
sigma = df['TEMP'].std(ddof=1) # <- nepristranska cenilka

# 95% napovedni interval za naslednji dan
#Ker sigma ni znan, moramo uporabiti Studentovo porazdelitev, vendar se pri 1114 prostostnih
# stopnjah kvantili Studentove porazdelitve že zelo ujemajo s kvantili N(0, 1).
# Ker privzemamo neodvisnost temperatur med dnevi, dobimo napovedni interval za temperaturo naslednji dan.
sp_meja = mu - 1.96 * sigma * np.sqrt(1 + 1/n)
zg_meja = mu + 1.96 * sigma * np.sqrt(1 + 1/n)
interval = (sp_meja, zg_meja)
print(f"povprečje: {mu}")
print(f"standardni odklon: {sigma}")
print(f"95% napovedni interval (beli šum): {interval}")

povprečje: 14.090421524663677
standardni odklon: 12.910692967851453
95% napovedni interval (beli šum): (np.float64(-11.22588166407513), np.float64(39.40672471340248))


In [65]:
from scipy.optimize import curve_fit
import numpy as np

df = pd.read_csv('Temp_MN.csv')

dan = df['DAN']

def model(t, a0, a1, a2):
    return a0 + a1 * np.sin(2 * np.pi * t / 365.25) + a2 * np.cos(2 * np.pi * t / 365.25)

# Prileganje modela
# popt ... optimalni parametri
# pcov ... kovariančna matrika
popt, _ = curve_fit(model, dan, df['TEMP'])
sumi = df['TEMP'] - model(dan, *popt) # zvezdica 'razpakira' podatke
povprecni_sum = sumi.mean()
sigma = sumi.std(ddof=1)

# Napoved za naslednji dan (zadnji dan + 1)
zadnji_dan = df['DAN'].iloc[-1]
naslednji_dan = zadnji_dan + 1
napoved = model(naslednji_dan, *popt) # zvezdica 'razpakira' podatke


# 95% napovedni interval za naslednji dan
#Ker sigma ni znan, moramo uporabiti Studentovo porazdelitev, vendar se pri 1114 prostostnih
# stopnjah kvantili Studentove porazdelitve že zelo ujemajo s kvantili N(0, 1).
# Ker privzemamo neodvisnost Šumov med dnevi, dobimo napovedni interval za temperaturo naslednji dan.
sp_meja1 = napoved + povprecni_sum - 1.96 * sigma * np.sqrt(1 + 1/n)
zg_meja1 = napoved + povprecni_sum + 1.96 * sigma * np.sqrt(1 + 1/n)
interval = (sp_meja1, zg_meja1)
print(f"povprečni šum: {povprecni_sum}")
print(f"opimalni parametri: {popt}")
print(f"95% napovedni interval (sezonski model): {interval}")

povprečni šum: 1.804698274707481e-08
opimalni parametri: [12.29655081  7.298332   14.36784202]
95% napovedni interval (sezonski model): (np.float64(6.2064988255939095), np.float64(28.935894566793415))


In [66]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import norm

# Naloži podatke
df = pd.read_csv('Temp_MN.csv')

# Poskrbi, da so podatki razvrščeni po dnevu
df = df.sort_values('DAN')

# Izračunaj spremembe temperatur med zaporednimi dnevi (delta_T)
df['delta_T'] = df['TEMP'].diff()
df['delta_DAN'] = df['DAN'].diff()

# Uporabi le tiste vrstice, kjer je razlika dneva 1 (zaporedni dnevi)
mask = df['delta_DAN'] == 1
delta_T = df.loc[mask, 'delta_T']
dan_v_letu = ((df.loc[mask, 'DAN'] - 1) % 365) + 1

# Sezonski model za spremembe
def sezonska_sprememba(t, b1, b2):
    return b1 * np.sin(2 * np.pi * t / 365) + b2 * np.cos(2 * np.pi * t / 365) # CENTRIRANA: konstantni člen = 0

# Prileganje modela
popt, _ = curve_fit(sezonska_sprememba, dan_v_letu, delta_T)
ostanki = delta_T - sezonska_sprememba(dan_v_letu, *popt)
sigma = ostanki.std(ddof=1)

# Napoved za naslednji dan (1195)
naslednji_dan = 1195
naslednji_dan_v_letu = ((naslednji_dan - 1) % 365) + 1
delta_pred = sezonska_sprememba(naslednji_dan_v_letu, *popt)
T_pred = df['TEMP'].iloc[-1] + delta_pred

# 95% napovedni interval za naslednji dan
#Ker sigma ni znan, moramo uporabiti Studentovo porazdelitev, vendar se pri 1114 prostostnih
# stopnjah kvantili Studentove porazdelitve že zelo ujemajo s kvantili N(0, 1).
# Ker privzemamo neodvisnost sprememb temperatur med dnevi, dobimo napovedni interval za temperaturo naslednji dan.
sp_meja2 = T_pred - 1.96 * sigma * np.sqrt(1 + 1/n)
zg_meja2 = T_pred + 1.96 * sigma * np.sqrt(1 + 1/n)
interval = (sp_meja2, zg_meja2)
print(f"Napoved za dan {naslednji_dan} (dan v letu {naslednji_dan_v_letu}): {T_pred:.2f}")
print(f"95% napovedni interval (model sprememb): {interval}")


Napoved za dan 1195 (dan v letu 100): 8.03
95% napovedni interval (model sprememb): (np.float64(-0.7349512108824321), np.float64(16.797982487826363))
